# Transfer data from OLTP to OLAP
The idea is to join all tables from the OLTP database to transfer on table only to the OLAP database. OLAP database will have one time dimension.

### connect to oltp database and read the data

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from utils import connection

In [2]:
# get credentials for oltp schema
host, database, user, password = connection.mysql_params('oltp')

In [3]:
engine = create_engine(f"mysql://{user}:{password}@{host}:3306/{database}")
conn = engine.connect()

### query oltp tables changing names of the columns to match olap table

In [4]:
query = """
    SELECT
        User_ID,
        Name,
        Email_Address,
        Username,
        Date_of_Birth,
        Gender,
        l.location AS Location,
        Membership_Start_Date,
        Membership_End_Date,
        sp.subscription_plan AS Subscription_Plan,
        p.payment AS Payment_Information,
        r.renewal_status AS Renewal_Status,
        uf.usage_frequency AS Usage_Frequency,
        ph.purchase_history AS Purchase_History,
        g.genre AS Favorite_Genres,
        d.device AS Devices_Used,
        em.engagement_metrics AS Engagement_Metrics,  
        Feedback_Ratings,
        Customer_Support_Interactions
        FROM prime_users AS pu
        JOIN location AS l ON pu.Location_Id = l.location_id
        JOIN subscription_plan AS sp ON pu.Subscription_Plan_Id = sp.subscription_plan_id
        JOIN payment AS p ON pu.Payment_Info_Id = p.payment_id
        JOIN renewal AS r ON pu.Renewal_Status_Id = r.renewal_id
        JOIN usage_frequency AS uf ON pu.Usage_Frequency_Id = uf.usage_frequency_id
        LEFT JOIN purchase_history AS ph ON pu.Purchase_History_Id = ph.purchase_history_id
        JOIN genre AS g ON pu.Favorite_Genres_Id = g.genre_id
        JOIN device AS d ON pu.Devices_Used_Id = d.device_id
        JOIN engagement AS em ON pu.Engagement_Metrics_Id = em.engagement_id
"""

In [5]:
amazon = pd.read_sql(sql=query, con=conn)
amazon.shape

(2500, 19)

### Cargamos datos a OLAP

In [6]:
# get credentials for olap schema
host, database, user, password = connection.mysql_params('olap')

In [7]:
engine = create_engine(f"mysql://{user}:{password}@{host}:3306/{database}")
conn = engine.connect()

In [8]:
amazon.to_sql(name='prime_users', con=conn, if_exists='append', index=False)

2500